In [ ]:
import numpy as np
import pandas as pd
from astropy.stats import mad_std, sigma_clip
import re
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
from cytoolz import get_in
from astropy.time import Time
from datetime import datetime, timedelta
from pathlib import Path
import pickle

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.projects.sigma_circuits.segmentation as segmentation
import paulssonlab.projects.sigma_circuits.matriarch_stub as matriarch_stub
import paulssonlab.projects.sigma_circuits.experiment as experiment
import paulssonlab.io.metadata as metadata

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Load data

In [ ]:
# nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210324/PWM_flipped.nd2"
# nd2_filename2 = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210204/psc101_bistable.nd2"

In [ ]:
# nd2 = matriarch_stub.get_nd2_reader(nd2_filename)

In [ ]:
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210324flipped.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210204nonflipped_psc101.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210402_expt210331ramp.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210507_expt210505.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210514_expt210511.pickle"
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210605_expt210602_2.pickle"
data = pickle.load(open(filename, "rb"))
df = data["table"]
md = data["metadata"]
mux_log = data["mux_log"]
experiment_txt = data["experiment.txt"]
grid_df = data["grid"]

# Acquisition times

In [ ]:
md.keys()

In [ ]:
dtimeabsolute = get_in(
    ("SLxPictureMetadata", "dTimeAbsolute"),
    md["image_metadata_sequence"],
)
dtimeabsolute = Time(dtimeabsolute, format="jd").to_datetime()
dtimemsec = get_in(
    ("SLxPictureMetadata", "dTimeMSec"),
    md["image_metadata_sequence"],
)
tdelta = timedelta(milliseconds=dtimemsec)

In [ ]:
tdelta.total_seconds()

In [ ]:
first_position_times = md["acquisition_times"][:: len(grid_df)] / 1e3  # seconds

In [ ]:
last_good_cycle = np.argmax(np.diff(first_position_times) == 0) - 1

In [ ]:
seconds_per_timepoint = first_position_times[last_good_cycle] / last_good_cycle

In [ ]:
mux_times = [m[0] for m in mux_log]

In [ ]:
mux_timepoints = [
    (m - dtimeabsolute).total_seconds() / seconds_per_timepoint for m in mux_times
]

In [ ]:
mux_timepoints

In [ ]:
mux_timepoints2 = [
    np.argmax((m - dtimeabsolute).total_seconds() <= first_position_times)
    for m in mux_times
]

In [ ]:
mux_timepoints2

In [ ]:
plt.plot(md["acquisition_times"][:5000])

# Filtered

In [ ]:
%%time
cells = df.join(grid_df).reset_index()
cells.t = cells.t * 15 / 60
filtered_cells = cells[
    cells["area"].between(200, 800)
    & cells["GFP-PENTA"].between(300, 5000)
    & cells["row"].isin(["A", "B", "C", "D", "F", "G", "H"])
    #     & cells["RFP-PENTA"].between(800, 1200)
][["GFP-PENTA", "row", "t"]]

filtered_cells2 = filtered_cells.groupby(["row", "t"]).apply(
    lambda y: sigma_clip(y, sigma=2, maxiters=10, masked=False)
)

means = filtered_cells2.apply(np.mean)
sigmas = filtered_cells2.apply(np.std)

limits = pd.DataFrame({"lower": means - sigmas, "upper": means + sigmas})

In [ ]:
mean_plot = means.hvplot.line(
    "t", by="row", logy=True, ylim=[600, 5000], xlim=[9.25, 24]
)
noise_plot = limits.hvplot.area(
    x="t", y="lower", y2="upper", by="row", stacked=False, alpha=0.2, logy=True
)
# mux_events = hv.Overlay(
#     [
#         hv.VLine(t).opts(alpha=0.2, color="grey", line_dash="dotted")
#         for t in mux_timepoints
#     ]
# )

# line_alpha = 0.2
# IPTG_starts = (
#     hv.VLine(12).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(60).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(108).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(156).opts(alpha=line_alpha, color="blue")
# )
# EZ_starts = (
#     hv.VLine(30).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(78).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(126).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(174).opts(alpha=line_alpha, color="grey")
# )
# aTc_starts = (
#     hv.VLine(42).opts(alpha=line_alpha, color="red")
#     * hv.VLine(90).opts(alpha=line_alpha, color="red")
#     * hv.VLine(138).opts(alpha=line_alpha, color="red")
#     * hv.VLine(186).opts(alpha=line_alpha, color="red")
# )

# (IPTG_starts*EZ_starts*aTc_starts*means*errors).opts(width=1000,height=400)
# (means * errors).opts(width=1000, height=400)

In [ ]:
(mean_plot * noise_plot).opts(
    width=1000, height=400, ylabel="GFP Fluorescence", xlabel="Hours", logy=True
)

In [ ]:
(sigmas / (means - 360)).hvplot.line(
    "t",
    by="row",
    logy=True,
    ylim=[0.05, 0.5],
    ylabel="Coefficient of Variation",
    xlabel="Time (Hours)",
    width=1000,
    height=400,
)

In [ ]:
from sklearn.linear_model import LinearRegression

for lane in ["A", "B", "C", "D", "E", "F", "G", "H"]:
    m = means[lane].reset_index()
    m_init = m[m["t"].between(3.5, 6)]
    X = m_init["t"].to_numpy().reshape(-1, 1)
    Y = m_init[0].to_numpy().reshape(-1, 1)
    linear_regressor = LinearRegression()  # create object for the class
    reg = linear_regressor.fit(X, Y)  # perform linear regression

    print(lane, reg.coef_, Y[0] - 360)
    plt.scatter(m["t"], m[0], linewidth=2)

    x_new = np.linspace(3, 10, 100)
    y_new = reg.predict(x_new[:, np.newaxis])
    plt.plot(x_new, y_new, linewidth=4, label=f"{lane}, {int(reg.coef_[0][0])}")
plt.xlim(2, 15)
plt.ylim(300, 4000)
plt.xlabel("Time (hours)")
plt.ylabel("GFP Fluorescence")
plt.rcParams.update({"font.size": 22})
# matplotlib.rc('xtick', labelsize=20)
# matplotlib.rc('ytick', labelsize=20)
plt.legend()
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression


def inv_hill(y):
    return y / (y.max() + 1 - y)


for lane in ["A", "B", "C", "D", "E", "F", "G", "H"]:
    m = means[lane].reset_index()
    m_init = m[m["t"].between(3.5, 15)]

    X = m_init["t"].to_numpy().reshape(-1, 1)
    Y = inv_hill(m_init[0]).to_numpy().reshape(-1, 1)
    linear_regressor = LinearRegression()  # create object for the class
    reg = linear_regressor.fit(X, Y)  # perform linear regression
    print(lane, reg.coef_)
    plt.scatter(X, Y, linewidth=2)

    x_new = np.linspace(3, 10, 100)
    y_new = reg.predict(x_new[:, np.newaxis])
#     plt.plot(x_new,y_new,linewidth=4,label=f'{lane}, {int(reg.coef_[0][0])}')
# plt.xlim(0,20)
plt.ylim(0, 50)
plt.xlabel("Time (hours)")
plt.ylabel("GFP Fluorescence")
plt.rcParams.update({"font.size": 22})
# matplotlib.rc('xtick', labelsize=20)
# matplotlib.rc('ytick', labelsize=20)
plt.legend()
plt.show()

In [ ]:
(mean_plot * noise_plot * mux_events).opts(width=800, height=400)

In [ ]:
print(experiment_txt)

In [ ]:
mux_log

In [ ]:
mux_timepoints